In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
from __future__ import division
sys.path.append(r"C:\dev\opinion\opinion\python\opinion")
import os
import matplotlib.pyplot as plt

import utils.fileutils as fileutils
import utils.nlputils as nlputils
import utils.metrics as metrics

import cPickle as pickle
import pandas as pd
import numpy as np
import scipy as sp

from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr
from math import sqrt
from dateutil import parser
import datetime
from math import sqrt

In [2]:
indir = r"C:\dev\opinion\papers\covid19\CovidDevRantOpenCoding"
infileXL = os.path.join(indir, "CovidTagContentsWithCategoryMajorRevisionWithFinalCategories.xlsx")
infilePKL = os.path.join(indir, "CovidTagContentsWithCategoryMajorRevisionWithFinalCategories.pkl")

def readPKL(infile):
    with open(infile, 'rb') as f:
        df = pickle.load(f)
    return df

In [3]:
#df = pd.read_excel(infileXL)
df = readPKL(infilePKL)
df.head(2)

,id,creation_time,score,num_comments,tags,has_image,text,Relevant,Label1,Label2,...,Emotion2,Emotion3,sarcasm,irony,Category1,Category2,Category3,Category4,FinalCategory4,FinalCategory3
0,2438510,2020-03-21 15:17:25,106,17,rant,NaN,Clickbait will never change. \nIt's the most s...,N,NaN,NaN,...,NaN,NaN,NaN,NaN,,,,,,
1,2423716,2020-03-05 07:31:51,95,10,"joke/meme,covid-19,corona,bug,ticket",NaN,COVID-19\nWhy it sounds like a JIRA ticket?,Y,joke/meme,bug,...,NaN,NaN,Y,NaN,,"joke/meme,coding & maintenance","entertainment,development","Wellbeing,Technical",Personal & Family Wellbeing|Technical Aspects,Development|Entertainment


In [4]:
df.columns

Index([            u'id',  u'creation_time',          u'score',
         u'num_comments',           u'tags',      u'has_image',
                 u'text',       u'Relevant',         u'Label1',
               u'Label2',         u'Label3',         u'Label4',
               u'Label5',         u'Label6',       u'polarity',
             u'Emotion1',       u'Emotion2',       u'Emotion3',
              u'sarcasm',          u'irony',      u'Category1',
            u'Category2',      u'Category3',      u'Category4',
       u'FinalCategory4', u'FinalCategory3'],
      dtype='object')

In [5]:
relevantTags = ['covid', 'covid-19', 'covid19', 'corona', 'coronavirus', 'corona virus', 
                'pandemic', 'quarantine', 'lockdown', '2019-ncov', 'pig flu', 'wk199', 
               'wk200', 'wk201', 'wk202', 'wk203', 'working from home', 'wfh', 'isolation', 
               'telework', 'video conferencing', 'apocalypse', 'bat attack', 'virus', 
               'covid-19 global hackathon', 'jaicorona', 'random', 'question', 'devrant', 
               'joke/meme', 'rant']
print(len(relevantTags))

31


In [6]:
# •	R4.C6. I would have liked to see a summary table with all tabs and related info e.g. 
# of posts, period from which the tag started to be used, percentage of relevant posts for each tag.
mindate = parser.parse('2020-01-29 00:00:00')
print mindate
def countPostsPerTagRelevant():
    count = dict()
    
    for tag in relevantTags: count[tag] = 0
    for index, row in df.iterrows():
        dp = row['creation_time']
        #dp = datetime.datetime.fromtimestamp(ctime)
        if dp < mindate: continue
        
        #if dp.month < 1 and dp.day < 29: continue
        #if dp.day < 29: continue
        
        tags = row['tags']
        #print tags
        tags = tags.split(',')
        isRelevant = row['Relevant']
        if isRelevant == 'Y': 
            for tag in tags:
                if tag in count: 
                    count[tag] += 1
    return count

def countPostsPerTagAll():
    count = dict()
    for tag in relevantTags: count[tag] = 0
    for index, row in df.iterrows():
        dp = row['creation_time']
        #dp = datetime.datetime.fromtimestamp(ctime)
        if dp < mindate: continue
        tags = row['tags']
        #print tags
        tags = tags.split(',')
        for tag in tags:
            if tag in count: 
                count[tag] += 1
    return count

def getTagEarliestDate():
    dates = dict()
    for tag in relevantTags: dates[tag] = None
    for index, row in df.iterrows():
        isRelevant = row['Relevant']
        if isRelevant != 'Y': continue
        dp = row['creation_time']
        if dp < mindate: continue
        #print dp
        tags = row['tags']
        tags = tags.split(',')
        for tag in tags:
            if tag in dates:
                if dates[tag] is None:
                    dates[tag] = dp
                elif dates[tag] > dp:
                    dates[tag] = dp
    return dates

# •	R4.C7. It would be interesting to have some analysis of *other* tags 
# (e.g., related to techniques, domains, etc.) in the posts you analyzed. It can be mentioned as an 
# additional possibility for future research
def findOtherTags():
    count = dict()
    for index, row in df.iterrows():
        dp = row['creation_time']
        #dp = datetime.datetime.fromtimestamp(ctime)
        if dp < mindate: continue
        tags = row['tags']
        #print tags
        tags = tags.split(',')
        found = 0
        for tag in tags:
            if tag in relevantTags:
                found = 1
                break
        if found:
            for tag in tags:
                if tag not in relevantTags:
                    if tag not in count: 
                        count[tag] = 1
                    else:
                        count[tag] += 1
    return count

def computeNewNegativityRatioPerCategory():
    pass

2020-01-29 00:00:00


In [7]:
countRelevant = countPostsPerTagRelevant()
print(countRelevant)

{'corona virus': 36, 'pig flu': 3, 'random': 107, 'covid-19 global hackathon': 3, 'bat attack': 3, 'covid-19': 142, 'wk199': 162, 'wk201': 13, 'wk200': 30, 'wk203': 21, 'wk202': 0, 'lockdown': 30, 'video conferencing': 2, 'question': 47, 'covid19': 36, 'corona': 57, '2019-ncov': 4, 'jaicorona': 4, 'joke/meme': 116, 'devrant': 24, 'working from home': 10, 'virus': 7, 'telework': 3, 'wfh': 56, 'rant': 544, 'isolation': 5, 'quarantine': 49, 'pandemic': 11, 'covid': 26, 'coronavirus': 119, 'apocalypse': 6}


In [132]:
countRelevant = countPostsPerTagRelevant()
countAll = countPostsPerTagAll()
dates = getTagEarliestDate()
sortedCountAll = sorted(countAll.items(), key=lambda item: item[1], reverse=True)
for item in sortedCountAll:
    #print item
    k = item[0]
    #print k
    vall = countAll[k]
    vrell = countRelevant[k]
    #if vall != 0:
    if vall != 0:
        ratio = vrell/vall
        print k, countAll[k], countRelevant[k], '%.4f'%(ratio), dates[k]

rant 590 544 0.9220 2020-01-29 01:14:42
wk199 170 162 0.9529 2020-03-09 00:41:18
covid-19 162 142 0.8765 2020-03-05 07:31:51
coronavirus 145 119 0.8207 2020-03-08 01:08:14
joke/meme 135 116 0.8593 2020-02-01 14:10:23
random 127 107 0.8425 2020-01-30 23:17:26
corona 72 57 0.7917 2020-02-01 14:10:23
wfh 58 56 0.9655 2020-03-10 10:32:03
question 53 47 0.8868 2020-02-13 13:31:30
quarantine 52 49 0.9423 2020-03-15 14:52:34
corona virus 48 36 0.7500 2020-02-08 22:36:26
covid19 43 36 0.8372 2020-03-09 02:13:11
lockdown 34 30 0.8824 2020-03-14 21:07:30
wk200 31 30 0.9677 2020-03-16 07:24:56
covid 30 26 0.8667 2020-03-13 10:06:18
devrant 27 24 0.8889 2020-03-13 10:43:47
wk203 21 21 1.0000 2020-04-06 01:15:26
wk201 13 13 1.0000 2020-03-23 01:05:32
working from home 11 10 0.9091 2020-03-17 06:18:31
pandemic 11 11 1.0000 2020-03-19 18:25:21
apocalypse 9 6 0.6667 2020-03-13 12:51:22
virus 8 7 0.8750 2020-01-31 00:08:39
bat attack 5 3 0.6000 2020-03-17 16:44:57
isolation 5 5 1.0000 2020-03-29 12:42:

In [103]:
def getRantsForTagOnDate(atag, adate):
    for index, row in df.iterrows():
        dp = row['creation_time']
        if dp.year < 2020: continue
        
        if dp.year!=int(adate.year): continue
        if dp.month!=int(adate.month): continue
        if dp.day!=int(adate.day): continue
        print dp.year, dp.month, dp.day
        isRelevant = row['Relevant']
        if isRelevant != 'Y': continue
        tags = row['tags']
        tags = tags.split(',')
        for tag in tags:
            if tag == atag:
                print row['Relevant'], row['text'], tags, dp
                print '==================='

In [133]:

adate = '2020-01-29 01:14:00'
adate = parser.parse(adate)
print adate.year, adate.month, adate.day
atag = 'rant'
getRantsForTagOnDate(atag, adate)

2020 1 29
2020 1 29
Y Gets severe vertigo when laying down starting yesterday. Most likely because I worked 110 hours last week.

Brain: “coronavirus, panic”

Not even a hypochondriac but even I’m a little on edge because of the plague. [u'rant', u'!dev'] 2020-01-29 01:14:42


In [137]:
countOther = findOtherTags()
print(len(countOther))
sortedCountOther= sorted(countOther.items(), key=lambda item: item[1], reverse=True)
sortedCountOther

525


[(u'remote work', 18),
 (u'joke', 15),
 (u'work from home', 15),
 (u'meme', 12),
 (u'work', 11),
 (u'developer', 10),
 (u'memes', 10),
 (u'fuck', 9),
 (u'remote', 9),
 (u'vpn', 7),
 (u'dev', 7),
 (u'github', 7),
 (u'python', 7),
 (u'windows', 7),
 (u'wtf', 7),
 (u'funny', 6),
 (u'android', 6),
 (u'php', 6),
 (u'home office', 6),
 (u'linux', 6),
 (u'home', 6),
 (u'git', 6),
 (u'corona antivirus', 5),
 (u'jira', 5),
 (u'programming', 5),
 (u'sweden', 5),
 (u'germany', 5),
 (u'pffft', 5),
 (u'chatroulette', 5),
 (u'i hate my wife', 5),
 (u'uni', 5),
 (u'airlines', 5),
 (u'help please', 5),
 (u'china', 4),
 (u'tesseract', 4),
 (u'fuckcorona', 4),
 (u'end of the world', 4),
 (u's3', 4),
 (u'who knows this might work', 4),
 (u'google cloud vision', 4),
 (u'hackathon', 4),
 (u'#covid #corona #mentalhealth', 4),
 (u'research', 4),
 (u'less distractions', 4),
 (u'less screens', 4),
 (u'workfromhome', 4),
 (u'slack', 4),
 (u'stock market', 4),
 (u'future', 4),
 (u'who', 4),
 (u'medical', 4),
 (u